# Project 2: Subsistence Diets

In [141]:
apikey = "E6haYn1Fyu8cDrLwizdiVB0pPh1FqIlqdEze5lfw" 

In [142]:
# %pip install pandas

import pandas as pd
import numpy as np
import warnings
import requests

### [A] Dietary Reference Intakes Function

Write a function that takes as arguments the characteristics of a person (e.g., age, sex) and returns a `pandas.Series' of Dietary Reference Intakes (DRI's) or "Recommended Daily Allowances" (RDA) of a variety of nutrients appropriate for your population of interest.

In [143]:
rda = pd.read_csv("rda.csv", index_col = 0)

options = ['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 'Female_14_18', 'Male_14_18', 'Female_19_30', 'Male_19_30', 'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']

bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI'])]
bmax = rda.loc[rda['Constraint Type'].isin(['UL'])]

In [144]:
def dietary_ref_intake(age = 20,sex = "Female", data = rda):
    """Takes in age (integer) and sex (string), and returns a Series of dietary reference intakes for the chosen population, you can optionally use a different data frame, the min or max RDAs"""

    if age <= 3:
        col = 'Child_1_3'
    if sex in ["M", "male", "m"]:
        sex = "Male"
    elif sex in ["F", "f", "female"]:
        sex = "Female"
    if age <= 3:
        col = 'Child_1_3'
    elif age >= 51:
        col = sex + "_51U" 
    else:
        age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(51,100)]
        for age_range in age_ranges:
            if age >= age_range[0] and age <= age_range[1]:
                col = sex + '_' + str(age_range[0]) + '_' + str(age_range[1])
    return pd.Series(data[col])  

#### Examples

In [145]:
dietary_ref_intake(age=22,sex='M')

Nutrient
Energy            2400.0
Protein             56.0
Carbohydrate       130.0
Dietary Fiber       33.6
Linoleic Acid       17.0
Linolenic Acid       1.6
Calcium           1000.0
Iron                 8.0
Magnesium          400.0
Phosphorus         700.0
Potassium         4700.0
Sodium_max        2300.0
Zinc                11.0
Copper               0.9
Selenium            55.0
Vitamin A          900.0
Vitamin E           15.0
Vitamin D           15.0
Vitamin C           90.0
Thiamin              1.2
Riboflavin           1.3
Niacin              16.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            550.0
Vitamin K          120.0
Folate             400.0
Energy_max        3100.0
Name: Male_19_30, dtype: float64

In [113]:
dietary_ref_intake(age=80,sex='F', data = bmax)

Nutrient
Sodium_max    2300.0
Energy_max    3100.0
Name: Female_51U, dtype: float64

### [A] Data on Prices for Different Foods

Construct a google spreadsheet of the prices of different food products for each diet (frozen food diet, meat diet, fresh food diet, liquid diet, and canned-food diet)

In [149]:
# Define file paths again if they are not available
file_paths = {
    "carnivore": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - carnivore_recipes.csv",
    "canned": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - canned_recipes.csv",
    "frozen": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - frozen_recipes.csv",
    "fresh": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - fresh_recipes.csv",
    "liquid": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - liquid_recipes.csv",
    "prices": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - prices.csv",
}


# Function to read a dataset
def read_sheet(file_path):
    df = pd.read_csv(file_path, index_col=False)
    df = df.iloc[:, :7].dropna(subset=['parent_foodcode'])
    df = df.reset_index(drop=True)
    return df

# Load prices dataset
prices_df = pd.read_csv(file_paths["prices"])
prices_df['parent_foodcode'] = prices_df['parent_foodcode'].astype(int)  # Convert type for merging

# Function to merge price with a given diet dataset
def read_and_merge_with_prices(diet_name):
    df = read_sheet(file_paths[diet_name])  # Read the diet dataset
    df['parent_foodcode'] = df['parent_foodcode'].astype(int)  # Ensure data type matches for merging
    merged_df = df.merge(prices_df, on="parent_foodcode", how="left")  # Left join to include all diet rows
    return merged_df

# Now run the function without errors
frozen_diet_with_prices = read_and_merge_with_prices("frozen")


In [150]:
#Example of merged diet and price
frozen_diet_with_prices.head()

,parent_foodcode,parent_desc,ingred_code,ingred_desc,ingred_wt,year,mod_code,method,method_description,nhanes,price
0,11460150,"Yogurt, frozen, NS as to flavor, lowfat milk",1298,"Yogurt, frozen, flavors other than chocolate, ...",100.0,2013/2014,NaN,2.0,Links to altEC,Extra,0.335298
1,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8,2011/2012,0.0,1.0,Links to FNDDS,NaN,0.276580
2,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8,2013/2014,NaN,1.0,Links to FNDDS,Extra,0.296941
3,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8,2015/2016,NaN,1.0,Links to FNDDS,Extra,0.301143
4,11460160,"Yogurt, frozen, chocolate, lowfat milk",19166,"Cocoa, dry powder, unsweetened, processed with...",5.2,2011/2012,0.0,1.0,Links to FNDDS,NaN,0.276580


### [A] Nutritional Content of Different Foods

Write a function that describes the nutritional content for each diet.

In [151]:
def read_nutrients():
    """
    Reads the nutrients dataset and ensures column formatting is correct.
    """
    nutrients_df = pd.read_csv("~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - nutrients.csv", index_col=False)

    # Strip any spaces from column names to avoid merge issues
    nutrients_df.columns = nutrients_df.columns.str.strip()

    # Print columns for debugging
    print("Nutrients dataset columns:", nutrients_df.columns)

    return nutrients_df

# Load the nutrients dataset once
nutrients_df = read_nutrients()


Nutrients dataset columns: Index(['ingred_code', 'Ingredient description', 'Capric acid', 'Lauric acid',
       'Myristic acid', 'Palmitic acid', 'Palmitoleic acid', 'Stearic acid',
       'Oleic acid', 'Linoleic Acid', 'Linolenic Acid', 'Stearidonic acid',
       'Eicosenoic acid', 'Arachidonic acid', 'Eicosapentaenoic acid',
       'Erucic acid', 'Docosapentaenoic acid', 'Docosahexaenoic acid',
       'Butyric acid', 'Caproic acid', 'Caprylic acid', 'Alcohol', 'Caffeine',
       'Calcium', 'Carbohydrate', 'Carotene, alpha', 'Carotene, beta',
       'Cholesterol', 'Choline', 'Copper', 'Cryptoxanthin, beta', 'Energy',
       'Fatty acids, total monounsaturated',
       'Fatty acids, total polyunsaturated', 'Fatty acids, total saturated',
       'Dietary Fiber', 'Folate, DFE', 'Folate, food', 'Folate', 'Folic acid',
       'Iron', 'Lutein + zeaxanthin', 'Lycopene', 'Magnesium', 'Niacin',
       'Phosphorus', 'Potassium', 'Protein', 'Retinol', 'Riboflavin',
       'Selenium', 'Sodium', '

In [156]:
def get_diet_nutritional_info(diet_name, nutrients_df):
    """
    Fetches nutrient information for foods in a specified diet by using the ingred_code
    and merging with an existing nutrients dataset.

    Parameters:
        diet_name (str): The diet category (e.g., "frozen", "canned").
        nutrients_df (pd.DataFrame): The dataset containing nutrient information for each ingred_code.

    Returns:
        pd.DataFrame: Nutritional content for the diet, formatted with:
                      - Rows as nutrients (e.g., Protein, Zinc, Water).
                      - Columns as food items in the diet.
    """
    # Read the specific diet dataset
    diet_df = read_sheet(file_paths[diet_name])  

    # Ensure `ingred_code` exists in both datasets
    if "ingred_code" not in diet_df.columns:
        raise ValueError(f"Column 'ingred_code' not found in {diet_name} dataset.")
    
    if "ingred_code" not in nutrients_df.columns:
        raise ValueError("Column 'ingred_code' not found in nutrients dataset.")

    # Merge diet data with nutrient information using `ingred_code`
    merged_df = diet_df.merge(nutrients_df, on="ingred_code", how="left")

    # Add a column for the diet name
    merged_df["Diet"] = diet_name  

    # Pivot the table: Rows = Nutrients, Columns = Food Items
    nutrient_table = merged_df.set_index(["Diet", "parent_desc"]).drop(columns=["ingred_code"]).T

    return nutrient_table


In [ ]:
##example for frozen
nutritional_info_df = get_diet_nutritional_info("frozen", nutrients_df)

# Show first 10 rows to verify the diet name
nutritional_info_df.head(10)


Diet                                                               frozen                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     \
parent_desc                  Yogurt, frozen, NS as to flavor, lowfat milk             Yogurt, frozen, chocolate, lowfat milk             Yogurt, frozen, chocolate, lowfat milk Yogurt, frozen, chocolate, lowfat milk Yogurt, frozen, flavors other than chocolate, lowfat milk       Yogurt, frozen, NS as to flavor, nonfat milk Yogurt, frozen, NS as to flavor, nonfat milk       Yogurt, frozen, NS as to flavor, nonfat milk             Yogurt, frozen, chocolate, nonfat milk Yogurt, frozen, flavors other than chocolate, with sorbet or sorbet-coated Yogurt, frozen, flavors other than chocolate, with sorbet or sorbet-coated Yogurt, frozen, flavors other than chocolate, nonfat milk Yogurt, frozen, chocolate, nonfat milk, with low-calorie sweetener Yogurt, frozen, flavors other than chocolate, nonfat milk, with low-calorie sweetener   Yogurt, frozen, NS as to flavor, whole milk Yogurt, frozen, chocolate, whole milk Yogurt, frozen, flavors other than chocolate, whole milk   
parent_foodcode                                                  11460150                                           11460160                                           11460160                               11460160                                           11460170                                                  11460190                                     11460190                                           11460190                                           11460200                                           11460250                                                                   11460250                                                                   11460300                                                  11460400                                                           11460410                                                                         11460420                              11460430                                           11460440         
ingred_desc             Yogurt, frozen, flavors other than chocolate, ...  Yogurt, plain, low fat, 12 grams protein per 8...  Cocoa, dry powder, unsweetened, processed with...                     Sugars, granulated  Yogurt, frozen, flavors other than chocolate, ...         Yogurt, plain, skim milk, 13 grams protein per...                           Sugars, granulated  Frozen yogurts, chocolate, nonfat milk, sweete...  Frozen yogurts, chocolate, nonfat milk, sweete...                         Ice creams, vanilla, light                                     Frozen novelties, fruit and juice bars                          Yogurt, frozen, flavors not chocolate, nonfat ...         Frozen yogurts, chocolate, nonfat milk, sweete...                  Yogurt, frozen, flavors not chocolate, nonfat ...                                     Frozen yogurts, flavors other than chocolate             Frozen yogurts, chocolate       Frozen yogurts, flavors other than chocolate         
ingred_wt                                                        100.0000                       

# Liquid Diet Solution

In [63]:
liquid_recipes = pd.read_csv("Wilbur Atwater min_cost_data - liquid_recipes.csv")
nutrition = pd.read_csv("Wilbur Atwater min_cost_data - nutrients.csv")
# from fndds diet problem: normalize weights to percentage terms. 
liquid_recipes['ingred_wt'] = liquid_recipes['ingred_wt']/liquid_recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
liquid_df = liquid_recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(liquid_df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
liquid_df[numeric_cols] = liquid_df[numeric_cols].mul(liquid_df["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. 
#The ** effectively "unpacks" the key value pairs in each dictionary
liquid_df = liquid_df.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "parent_desc": "first"})

liquid_df.index.name = "recipe_id"

food_names = liquid_df["parent_desc"]
print(food_names.head())
liquid_df.head()

recipe_id
353.309772                                   Fruit smoothie, NFS
353.903556            Fruit smoothie, with whole fruit and dairy
382.332989     Fruit smoothie, with whole fruit and dairy, ad...
1258.921519                  Yogurt parfait, low fat, with fruit
1262.662207                        Yogurt, whole milk, baby food
Name: parent_desc, dtype: object


,parent_foodcode,ingred_code,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc,parent_desc
recipe_id,,,,,,,,,,,,,,,,,,,,,
353.309772,353.309772,30.560832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.305814,0.0,0.0,0.0,0.0,0.0,"Fruit smoothie, NFS"
353.903556,353.903556,30.612167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.306328,0.0,0.0,0.0,0.0,0.0,"Fruit smoothie, with whole fruit and dairy"
382.332989,382.332989,33.071245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.330935,0.0,0.0,0.0,0.0,0.0,"Fruit smoothie, with whole fruit and dairy, ad..."
1258.921519,1258.921519,109.913989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.099879,0.0,0.0,0.0,0.0,0.0,"Yogurt parfait, low fat, with fruit"
1262.662207,1262.662207,109.913989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.099879,0.0,0.0,0.0,0.0,0.0,"Yogurt, whole milk, baby food"


In [64]:
prices_liquid = prices_df[["parent_foodcode", "year", "price"]]



prices_liquid = prices_liquid.set_index(["year", "parent_foodcode"])
print(prices_liquid.index.levels[0])

# we'll focus on the latest price data
prices_liquid = prices_liquid.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices_liquid = prices_liquid.dropna(subset="price")
common_recipes = liquid_df.index.intersection(prices_liquid.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 
liquid_df = liquid_df.loc[common_recipes]
prices_liquid = prices_liquid.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices_liquid.index = prices_liquid.index.map(food_names)
A_liquid_all = liquid_df.T


print(f"We have prices for {prices_liquid.shape[0]} unique recipes (FNDDS food codes)")

Index(['2011/2012', '2013/2014', '2015/2016', '2017/2018'], dtype='object', name='year')
We have prices for 381 unique recipes (FNDDS food codes)


In [65]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''
group = "Female_19_30"

# create lower bounds and upper bounds.
bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group]
bmax = rda.loc[rda['Constraint Type'].isin(['UL']), group]

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_liquid_all.reindex(bmin.index).dropna(how='all')
Amax = A_liquid_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices_liquid.shape=}")

bmin.shape=(26,)
Amin.shape=(26, 381)
bmax.shape=(2,)
Amax.shape=(2, 381)
b.shape=(28,)
A.shape=(28, 381)
prices_liquid.shape=(381, 1)


In [66]:
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''

group = 'Female_19_30'
tol = 1e-6

result = lp(prices_liquid, -A, -b, method="highs")
result
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")
diet = pd.Series(result.x,index=prices_liquid.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(round(diet[diet >= tol], 2))

Cost of diet for Female_19_30 is $4.63 per day.

You'll be eating (in 100s of grams or milliliters):
Soy milk, light, chocolate                                                              20.17
Orange juice, 100%,  freshly squeezed                                                    0.72
Vegetable noodle soup, reduced sodium, canned, prepared with water or ready-to-serve     5.72
Corn oil                                                                                 0.75
Molasses                                                                                 0.06
Tea, iced, brewed, black, unsweetened                                                   11.20
Nutritional powder mix, high protein (Herbalife)                                         0.09
dtype: float64


In [67]:
tab = pd.DataFrame({"Outcome":A.to_numpy()@diet.to_numpy(),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)


With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2000.0          2000.0
Protein            58.93592            46.0
Carbohydrate      234.71626           130.0
Dietary Fiber          28.0            28.0
Linoleic Acid      48.96892            12.0
Linolenic Acid     2.092766             1.1
Calcium         2676.872671          1000.0
Iron              18.613537            18.0
Magnesium        476.001765           310.0
Phosphorus      2098.015412           700.0
Potassium            4700.0          4700.0
Zinc               8.368895             8.0
Copper             4.877076             0.9
Selenium               55.0            55.0
Vitamin A       1588.448858           700.0
Vitamin E         18.783996            15.0
Vitamin D         25.281843            15.0
Vitamin C              75.0            75.0
Thiamin            1.342018             1.1
Riboflavin         4.6

In [68]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())


Constraining nutrients are:
['Energy', 'Dietary Fiber', 'Potassium', 'Selenium', 'Vitamin C', 'Folate']


# Fresh Diet Solution

In [50]:
fresh_recipes = pd.read_csv("Wilbur Atwater min_cost_data - fresh_recipes.csv")
nutrition = pd.read_csv("Wilbur Atwater min_cost_data - nutrients.csv")
# from fndds diet problem: normalize weights to percentage terms. 
fresh_recipes['ingred_wt'] = fresh_recipes['ingred_wt']/fresh_recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
fresh_df = fresh_recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(fresh_df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
fresh_df[numeric_cols] = fresh_df[numeric_cols].mul(fresh_df["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. 
#The ** effectively "unpacks" the key value pairs in each dictionary
fresh_df = fresh_df.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "parent_desc": "first"})

fresh_df.index.name = "recipe_id"

food_names_2 = fresh_df["parent_desc"]
print(food_names_2.head())
fresh_df.head()

recipe_id
11169.565415                           Tuna, fresh, coated, fried
11199.028926     Tuna, fresh, coated, baked or broiled, fat added
11725.577014    Tuna, fresh, coated, baked or broiled, no adde...
17236.529516               Fresh corn custard, Puerto Rican style
89940.382826                          Onions, from fresh, creamed
Name: parent_desc, dtype: object


,parent_foodcode,ingred_code,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc,parent_desc
recipe_id,,,,,,,,,,,,,,,,,,,,,
11169.565415,11169.565415,0.866979,0.000015,0.000040,0.000013,0.000228,0.000033,0.000140,0.000276,0.000296,...,0.0,0.000124,0.000000,0.0,0.000444,0.0,0.069914,0.005321,0.000508,"Tuna, fresh, coated, fried"
11199.028926,11199.028926,0.869266,0.000015,0.000040,0.000013,0.000228,0.000033,0.000140,0.000277,0.000297,...,0.0,0.000125,0.000000,0.0,0.000445,0.0,0.070098,0.005335,0.000510,"Tuna, fresh, coated, baked or broiled, fat added"
11725.577014,11725.577014,0.910137,0.000016,0.000042,0.000013,0.000239,0.000035,0.000147,0.000290,0.000311,...,0.0,0.000130,0.000000,0.0,0.000466,0.0,0.073394,0.005586,0.000534,"Tuna, fresh, coated, baked or broiled, no adde..."
17236.529516,17236.529516,2.615980,0.000013,0.000013,0.000052,0.000522,0.000117,0.000144,0.009179,0.004073,...,0.0,0.000470,0.027399,0.0,0.003262,0.0,0.000000,0.012878,0.007176,"Fresh corn custard, Puerto Rican style"
89940.382826,89940.382826,2.441264,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000239,0.000119,"Onions, from fresh, creamed"


In [ ]:
prices_fresh = prices_df[["parent_foodcode", "year", "price"]]

prices_fresh = prices_fresh.set_index(["year", "parent_foodcode"])

# we'll focus on the latest price data
prices_fresh = prices_fresh.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices_fresh = prices_fresh.dropna(subset="price")
common_recipes = fresh_df.index.intersection(prices_fresh.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 

fresh_df = fresh_df.loc[common_recipes]
prices_fresh = prices_fresh.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices_fresh.index = prices_fresh.index.map(food_names_2)
A_fresh_all = fresh_df.T

print(f"We have prices for {prices_fresh.shape[0]} unique recipes (FNDDS food codes)")

We have prices for 7 unique recipes (FNDDS food codes)


,parent_foodcode,ingred_code,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc,parent_desc
22600200.0,22600200.0,10862.0,0.000,0.000,0.441,7.635,0.778,3.679,14.342,5.327,...,0.0,0.535,0.0,0.4,0.42,0.0,0.0,23.65,3.06,"Pork bacon, NS as to fresh, smoked or cured, c..."
22600210.0,22600210.0,43378.0,0.005,0.000,0.532,8.732,1.212,4.287,16.915,4.000,...,0.0,0.349,0.0,1.0,0.31,0.0,0.1,12.32,3.50,"Pork bacon, NS as to fresh, smoked or cured, r..."
22601040.0,22601040.0,10862.0,0.000,0.000,0.441,7.635,0.778,3.679,14.342,5.327,...,0.0,0.535,0.0,0.4,0.42,0.0,0.0,23.65,3.06,"Bacon or side pork, fresh, cooked"
26153100.0,26153100.0,15127.0,0.000,0.001,0.009,0.106,0.011,0.045,0.083,0.019,...,0.0,0.933,0.0,1.7,0.24,0.0,0.1,74.03,0.37,"Tuna, fresh, raw"
61204010.0,61204010.0,9152.0,0.020,0.001,0.002,0.012,0.000,0.004,0.004,0.012,...,0.0,0.046,38.7,0.0,0.15,0.0,0.0,92.31,0.05,"Lemon juice, 100%, freshly squeezed"
61207010.0,61207010.0,9160.0,0.000,0.000,0.000,0.008,0.002,0.000,0.007,0.015,...,0.0,0.038,30.0,0.0,0.22,0.0,0.6,90.79,0.08,"Lime juice, 100%, freshly squeezed"
61210010.0,61210010.0,9206.0,0.000,0.000,0.001,0.021,0.005,0.001,0.032,0.029,...,0.0,0.040,50.0,0.0,0.04,0.0,0.1,88.30,0.05,"Orange juice, 100%, freshly squeezed"


In [60]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''
group = "Female_19_30"

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_fresh_all.reindex(bmin.index).dropna(how='all')
Amax = A_fresh_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A_fresh = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices_fresh.shape=}")

bmin.shape=(26,)
Amin.shape=(26, 7)
bmax.shape=(2,)
Amax.shape=(2, 7)
b.shape=(28,)
A.shape=(28, 7)
prices_liquid.shape=(381, 1)


In [62]:
group = 'Female_19_30'
tol = 1e-6

result = lp(prices_fresh, -A_fresh, -b, method="highs")
result


       message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is None)
       success: False
        status: 2
           fun: None
             x: None
           nit: 0
         lower:  residual: None
                marginals: None
         upper:  residual: None
                marginals: None
         eqlin:  residual: None
                marginals: None
       ineqlin:  residual: None
                marginals: None

### Frozen Diet

In [ ]:
frozen_recipes = pd.read_csv("Wilbur Atwater min_cost_data - frozen_recipes.csv")
nutrition = pd.read_csv("Wilbur Atwater min_cost_data - nutrients.csv")
# from fndds diet problem: normalize weights to percentage terms. 
frozen_recipes['ingred_wt'] = frozen_recipes['ingred_wt']/frozen_recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
frozen_df = frozen_recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(frozen_df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
frozen_df[numeric_cols] = frozen_df[numeric_cols].mul(frozen_df["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. 
#The ** effectively "unpacks" the key value pairs in each dictionary
frozen_df = frozen_df.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "parent_desc": "first"})

frozen_df.index.name = "recipe_id"

food_names_2 = frozen_df["parent_desc"]
print(food_names_2.head())
frozen_df.head()

In [ ]:
prices_frozen = prices_df[["parent_foodcode", "year", "price"]]

prices_frozen = prices_frozen.set_index(["year", "parent_foodcode"])

# we'll focus on the latest price data
prices_frozen = prices_frozen.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices_frozen = prices_frozen.dropna(subset="price")
common_recipes = frozen_df.index.intersection(prices_frozen.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 

frozen_df = frozen_df.loc[common_recipes]
prices_frozen = prices_frozen.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices_frozen.index = prices_frozen.index.map(food_names_2)
A_frozen_all = frozen_df.T

print(f"We have prices for {prices_frozen.shape[0]} unique recipes (FNDDS food codes)")

In [ ]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''
group = "Female_19_30"

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_frozen_all.reindex(bmin.index).dropna(how='all')
Amax = A_frozen_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A_frozen = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices_frozen.shape=}")

In [ ]:
group = 'Female_19_30'
tol = 1e-6

result = lp(prices_frozen, -A_frozen, -b, method="highs")
result